<a href="https://colab.research.google.com/github/tcharos/Image-Classification-SVM/blob/main/SignalProcessing_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIDL Assignment 2 – Image Classification (MNIST)

Skeleton notebook for experimentation and implementation.
Fill in the code sections as required. Do **not** include answers here; analysis and results should go into the PDF report.

## 1. Imports and Global Settings

In [ ]:
import os
import requests
import tarfile
import numpy as np

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.datasets import mnist




## 2. Load and Inspect the MNIST Data

In [ ]:
# 1. Setup Directory
dataset_dir = './dataset'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    print(f"Created directory: {dataset_dir}")

# 2. Download Dataset
url = "http://www.di.ens.fr/~lelarge/MNIST.tar.gz"
target_path = os.path.join(dataset_dir, "MNIST.tar.gz")

In [ ]:
if not os.path.exists(target_path):
    print("Downloading dataset...")
    response = requests.get(url, stream=True)
    with open(target_path, "wb") as f:
        f.write(response.raw.read())
    print("Download complete.")

if os.path.exists(target_path):
    print("Extracting files...")
with tarfile.open(target_path, "r:gz") as tar:
    tar.extractall(path=dataset_dir)
print("Extraction complete.")

In [ ]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Inspect shapes and sample images
mnist.head()

## 3. Data Regrouping (Binary Classification)

In [ ]:
# Convert labels to binary classes:
# Class 1: digits 0–4, Class 2: digits 5–9


## 4. Preprocessing

In [ ]:
# Flatten images (28x28 -> 784)
# Optional normalization / scaling


## 5. SVM Benchmark on Raw Data

In [ ]:
# Train baseline SVM
# Evaluate total classification accuracy


## 6. SVM Hyperparameter Tuning

In [ ]:
# Grid search over C and kernels (linear, poly, rbf)
# Explore degree (poly) and gamma (rbf)


## 7. PCA Dimensionality Reduction

In [ ]:
# Apply PCA for different K values
# Train SVM on PCA-reduced data
# Store accuracy vs K


## 8. PCA Accuracy Plot

In [ ]:
# Plot SVM accuracy as a function of K


## 9. LDA Dimensionality Reduction

In [ ]:
# Apply LDA (1D output)
# Plot projected test data
# Train and evaluate SVM on LDA features


## 10. PCA + LDA

In [ ]:
# Apply PCA followed by LDA for different K
# Train SVM and record accuracy


## 11. Naïve Bayes Classifier

In [ ]:
# Train Gaussian Naïve Bayes
# Handle near-zero variance issue (e.g., var smoothing)
# Compare accuracy and computation time with SVM


## 12. Summary (No Answers Here)

In [ ]:
# This section may include brief code comments only.
# All discussion, plots, and conclusions go into the PDF report.
